
# Beyond CFT: Attacks on the Network and Convergence 


So far we showed how gossip can withstand network imperfection. But what if the attacker deliberately splits and attacks the network?  


Beyond regular crashes, peer can behave in various ways violating the protocol: hide transactions, send bogus data, create Sybil entities etc.
The goal of a blockchain system is to withstand against a powerful adversary. 

To ensure that message will be seen by the peer, once the peer is back online it must fetch the data from the neighboring peers. But what if the neighboring nodes are malicious and will censor certain transactions?   

In the next notebooks we will cover techniques that help to detect/prevent malicious behaviour.



# Malicious gossip agent 

One of the goal of a blockchain system is to record transaction in a 'hard-to-tamper' way.

How can you achieve that in P2P settings?  
It is common in databases and blockchains to use cryptography to verify the integrities of the transactions.

Let's first create a malicious agent that will change the data of received transactions to split the network.  


In [2]:
# Initialize the experiment:
import networkx as nx
import p2psimpy as p2p
import warnings
warnings.filterwarnings('ignore')

# Load the previous experiment configurations
exper = p2p.BaseSimulation.load_experiment(expr_dir='crash_gossip')

Locations, topology, peer_services, serv_impl = exper

print(peer_services['client'].service_map)


{'BaseConnectionManager': None, 'MessageProducer': None}


## Define malicious agents 
Let's first add malicious nodes randomly: 

In [27]:
# Change peer to a malicious 
from itertools import groupby
from random import sample

frac_malicious_nodes = 0.3 # 30 % of malicious nodes


def assign_malicious_peers(topology, mal_frac):
    print("mal_frac"+str(mal_frac))
    type_dict = nx.get_node_attributes(topology, 'type')
    inv_type_dict = {k: {j for j, _ in list(v)}
                                for k, v in groupby(type_dict.items(), lambda x: x[1])}
    mal_nodes = sample(list(inv_type_dict['peer']), 
                       int(mal_frac * len(inv_type_dict['peer'])))
    for b in mal_nodes:
        type_dict[b] = 'malicious'
        
    nx.set_node_attributes(topology, type_dict, 'type')
    
assign_malicious_peers(topology, frac_malicious_nodes)

mal_frac0.3


## Define malicious services 

We will inherit a malicious gossip service that will relay the gossip message to one half of the network and the other half a tempered message (with different data). 



In [28]:
from p2psimpy.messages import *
from p2psimpy.consts import TEMPERED

class MaliciousGossipService(p2p.GossipService):
    
    
    def handle_message(self, msg):
        # Store the original message localy 
        self.peer.store('msg_time', msg.id, self.peer.env.now)
        self.peer.store('msg_data', msg.id, msg.data)

        if msg.ttl > 0:
            # Rely message further, modify the message
            exclude_peers = {msg.sender} | self.exclude_peers
            
            # Send the original message to one half of the network, 
            selected = self.peer.gossip(GossipMessage(self.peer, msg.id, msg.data, msg.ttl-1,
                                                      pre_task=msg.pre_task, post_task=msg.post_task), 
                                        self.fanout//2, 
                                        except_peers=exclude_peers, 
                                        except_type=self.exclude_types)
            # Change the message and send it to the other half
            new_data = TEMPERED
            exclude_peers = exclude_peers | set(selected)
            self.peer.gossip(GossipMessage(self.peer, msg.id, new_data, msg.ttl-1, 
                                           pre_task=msg.pre_task, post_task=msg.post_task), 
                             self.fanout//2, 
                             except_peers=exclude_peers, 
                             except_type=self.exclude_types)

##  Add malicious type and services 

We deliberately keep malicious nodes uncrashable. 


In [29]:
gossip_config = peer_services['peer'].service_map['RangedPullGossipService']
serv_impl['RangedPullGossipService'] = p2p.GossipService



peer_services['malicious'] = p2p.PeerType(peer_services['peer'].config,
                                      {p2p.BaseConnectionManager:None,
                                       MaliciousGossipService: gossip_config}
                                     )

## Run simulation 

Let's see how malicious agents together with crashing nodes affect the message dissemination. 

In [30]:
serv_impl

{'BaseConnectionManager': p2psimpy.services.connection_manager.BaseConnectionManager,
 'MessageProducer': p2psimpy.services.message_producer.MessageProducer,
 'RandomDowntime': p2psimpy.services.disruption.RandomDowntime,
 'RangedPullGossipService': p2psimpy.services.gossip.GossipService}

In [31]:
from p2psimpy.messages import GossipMessage

In [32]:
# Init Graph
sim = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim.run(3_200)

# Analyze the storage data




## Message data

Let's see how this fraction of malicious nodes affected the network. 
We compare the received message with the original message, we report `True` if the message wasn't tampered `False` and otherwise. 



In [33]:
import pandas as pd

def message_data(sim, peer_id, storage_name):
    store = sim.peers[peer_id].storage[storage_name].txs
    for msg_id, tx in store.items():
        client_id, msg_num = msg_id.split('_')
        client_tx = sim.peers[int(client_id)].storage[storage_name].txs[msg_id]
        yield (int(msg_num), tx.data == client_tx.data)
        
def get_gossip_table(sim, storage_name, func):
    return pd.DataFrame({k: dict(func(sim, k, storage_name)) 
                         for k in set(sim.types_peers['peer'])}).sort_index()

    
df = get_gossip_table(sim, 'msg_data', message_data)
df

1      4      5      7      8     9     11     12    13     15     17  \
1   True   True   True   True   True  True  True  False  True   True   True   
2   True   True   True   True   True  True  True   True  True   True   True   
3   True   True   True   True  False  True  True   True  True   True   True   
4   True   True   True   True   True  True  True   True  True   True   True   
5   True   True   True   True  False  True  True  False  True   True   True   
6   True   True  False  False   True  True  True   True  True   True   True   
7   True   True  False  False   True  True  True   True  True   True   True   
8   True   True  False   True   True  True   NaN  False  True    NaN   True   
9    NaN  False   True   True    NaN  True  True   True  True   True   True   
10  True   True   True   True   True  True  True   True  True   True   True   
11  True   True   True   True   True  True  True   True  True  False  False   

      18    19    20    21    22     23     24  
1   True  True  True  True  True   True   True  
2   True  True  True  True  True  False   True  
3   True  True  True  True  True   True   True  
4   True  True  True  True  True   True   True  
5   True  True  True  True  True   True   True  
6   True  True  True  True  True  False   True  
7   True   NaN  True  True  True   True  False  
8   True  True  True  True  True   True    NaN  
9   True  True  True  True  True   True   True  
10  True  True  True  True  True   True  False  
11  True  True  True  True   NaN   True   True

In [34]:
df[df==False].count()

1     0
4     1
5     3
7     2
8     2
9     0
11    0
12    3
13    0
15    1
17    1
18    0
19    0
20    0
21    0
22    0
23    2
24    2
dtype: int64

Malicious nodes managed to trick some peers into accepting wrong data! As peers will write 'first-seen' value, adversary once having advantage over the network can perfectly split the network. 

How to deal with this?

# Signing messages

First of all, messages themselves must be verified on their **integrity** and **authenticity**. 
[Digital signatures](https://en.wikipedia.org/wiki/Digital_signature) are perfect match for this and hence all blockchain systems use them. 

We will modify the code to simulate the signed messages. We will not use an actual crytpographic protocol since we care about only two things for our simulation: 
- It takes time to verify and sign messages. 
- Peers should store and forward only valid messages. 



## Simulating digital signatures 

We will show an example by building a crypto validator for 456 bits [EdDSA (Ed448)](https://en.wikipedia.org/wiki/EdDSA) (one of the most popular digital signatures in the wild).
On a regular laptop it takes usually less than 1 millisecond to verify a signature. Let's take the near worse case.  


This is not real. 

We will integrate a verification task into the message itself. 
Peer before triggering other services will first run the `pre_task`.  

Since the message is first created by MessageProducer we will add a task in the configuration.



In [35]:
from p2psimpy.consts import TEMPERED
from p2psimpy.config import Config, Func, Dist
import random

conf = peer_services['client'].service_map['MessageProducer']


def validate_task(msg, peer):
    gen_dist = Dist('norm', (1, 0.2)) # time it takes to verify the message

    yield peer.env.timeout(gen_dist.get())
    if msg.data == TEMPERED:
        # You can decide what to do in this case.
        random_bit = random.uniform(0, 1)
        if random_bit <= 0.9:
            return False
        else:
            return True
    return True


class MsgConfig(Config):
    pre_task = Func(validate_task)
    init_ttl = conf.init_ttl if conf else 3
    
    
peer_services['client'].service_map['MessageProducer'] = MsgConfig

In [36]:
# Run the simulation with a modificed message producer 

sim2 = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim2.run(3_200)


In [37]:
df = get_gossip_table(sim2, 'msg_data', message_data)
df

1     4     5     7     8     9     11    12    13    15    17    18  \
1   True  True  True  True  True  True  True  True  True  True  True  True   
2    NaN  True  True  True  True  True  True  True  True  True  True  True   
3   True  True  True  True  True  True  True  True  True  True  True  True   
4   True  True  True  True  True  True  True  True  True  True  True  True   
5   True  True  True  True  True  True  True  True  True  True  True  True   
6   True  True  True  True  True  True  True  True  True  True  True   NaN   
7   True  True  True  True  True  True  True  True  True  True   NaN  True   
8   True   NaN  True  True  True  True  True  True  True  True  True  True   
9   True  True   NaN   NaN  True  True  True  True   NaN  True  True  True   
10  True  True  True  True  True  True  True  True  True  True  True  True   
11  True   NaN  True  True  True  True  True  True  True  True  True   NaN   

      19    20    21    22    23    24  
1   True  True  True  True  True  True  
2   True  True  True  True  True  True  
3   True  True  True  True  True  True  
4   True  True  True  True  True  True  
5   True  True  True  True  True  True  
6   True  True  True  True  True  True  
7   True  True  True  True   NaN  True  
8    NaN   NaN  True   NaN  True   NaN  
9   True  True  True  True   NaN  True  
10  True  True  True  True  True  True  
11  True  True  True  True  True  True

In [38]:
sim2.peers[22].storage['msg_data'].txs

{'26_1': GossipMessage:SUQCZGHBKCDDNVMIEAZS,
 '26_2': GossipMessage:CJTXVLXKRTRPAOPRFOEP,
 '26_3': GossipMessage:DHMSWWBEZADWQXYHVMIS,
 '26_4': GossipMessage:LBPJVAVGBMCCKICXUZHL,
 '26_5': GossipMessage:MMUNFVEQTVCJRHHRWUQS,
 '26_6': GossipMessage:LXNNSPCFGMOCTHKNZMVB,
 '26_7': GossipMessage:SNESOENUVQDNKQQCTYZQ,
 '26_9': GossipMessage:KNRBCUNBTHKMTERKCKRP,
 '26_10': GossipMessage:NXALLSGRJELTZHLTJKBK,
 '26_11': GossipMessage:PQKSOLSROUSKCBOSTKSH}

Now malicious nodes cannot change the message. They need to explore other attack strategies!
The malicious nodes can delay messages, hide them, freeriding in a gossip (only listening). 
Together with network attack this can create a dangerous combination. 

Let us consider a case where an honest node is surrounded by malicious nodes (all network connections are with malicious nodes) that will hide certain transactions. As a result, peer will not receive crucial transactions that might affect it's decision making process. This attack is also called **Eclipse attack**.   

In reality, almost nothing stops one malicious node from running multiple instances and poison the whole network. This attack is called **Sybil Attack**. 




### Exercises


- Explore the limits of the gossip protocol. What is the maximum number of malicious nodes a protocol can tolerate? 
- Try to eclipse attack some peer, make sure he doesn't get any message, or one specific message (censor)? 



In [17]:
# 1. don't attend gossip (e.g. stop sending)
# 2. try to send all data (e.g. double spending)

a)
The maximum number of malicious nodes a protocol can tolerate is related to the validation function. 

If the validation function can recognize all tampered message, even there are over 90% malicious nodes, honest node(s) can still recognize and disseminate true data. However, the honest node(s) will receive very few messages and the convergence can be poor. If the validation function can't recognize all tampered messages, it will be kind of vulnerable to malicious nodes. For example, if it can verify only half of the messages, even with only 10% malicious nodes, a honest node can end up receiving 13% false messages. 

b)
If one peer get eclipse attack, then all its neighbors are malicious nodes. Then the messages it received completely depends on how malicious nodes behave. For example, if malicious nodes do not send any messages, the poor peer will not recieve any messages at all.

In [13]:
import networkx as nx
import p2psimpy as p2p
import warnings
from itertools import groupby
from random import sample

warnings.filterwarnings('ignore')

# Load the previous experiment configurations
my_exper = p2p.BaseSimulation.load_experiment(expr_dir='crash_gossip')
my_locations, my_topology, my_peer_services, my_serv_impl = my_exper

In [14]:
def assign_malicious_peers(topology, mal_frac):
    print("mal_frac"+str(mal_frac))
    type_dict = nx.get_node_attributes(topology, 'type')
    inv_type_dict = {k: {j for j, _ in list(v)}
                                for k, v in groupby(type_dict.items(), lambda x: x[1])}
    mal_nodes = sample(list(inv_type_dict['peer']), 
                       int(mal_frac * len(inv_type_dict['peer'])))
    for b in mal_nodes:
        type_dict[b] = 'malicious'
        
    nx.set_node_attributes(topology, type_dict, 'type')
    
frac_malicious_nodes = 0.10
assign_malicious_peers(my_topology, frac_malicious_nodes)

mal_frac0.1


In [15]:
from p2psimpy.messages import *
from p2psimpy.consts import TEMPERED

class MyMaliciousGossipService(p2p.GossipService):
    
    
    def handle_message(self, msg):
        # Store the original message localy 
        self.peer.store('msg_time', msg.id, self.peer.env.now)
        self.peer.store('msg_data', msg.id, msg.data)

        if msg.ttl > 0:
            # Rely message further, modify the message
            exclude_peers = {msg.sender} | self.exclude_peers
            
            # Send the original message to one half of the network, 
#             selected = self.peer.gossip(GossipMessage(self.peer, msg.id, msg.data, msg.ttl-1,
#                                                       pre_task=msg.pre_task, post_task=msg.post_task), 
#                                         self.fanout//2, 
#                                         except_peers=exclude_peers, 
#                                         except_type=self.exclude_types)
#             # Change the message and send it to the other half
#             new_data = TEMPERED
#             exclude_peers = exclude_peers | set(selected)
#             self.peer.gossip(GossipMessage(self.peer, msg.id, new_data, msg.ttl-1, 
#                                            pre_task=msg.pre_task, post_task=msg.post_task), 
#                              self.fanout//2, 
#                              except_peers=exclude_peers, 
#                              except_type=self.exclude_types)

In [16]:
from p2psimpy.consts import TEMPERED
from p2psimpy.config import Config, Func, Dist
import random

my_conf = my_peer_services['client'].service_map['MessageProducer']

def my_validate_task(msg, peer):
    gen_dist = Dist('norm', (1, 0.2)) # time it takes to verify the message

    yield peer.env.timeout(gen_dist.get())
    if msg.data == TEMPERED:
        # You can decide what to do in this case.
        random_bit = random.uniform(0, 1)
        if random_bit <= 0.5:
            return False
        else:
            return True
    return True


class MsgConfig(Config):
    pre_task = Func(my_validate_task)
    init_ttl = my_conf.init_ttl if my_conf else 3
    
    
my_peer_services['client'].service_map['MessageProducer'] = MsgConfig

In [17]:


my_gossip_config = my_peer_services['peer'].service_map['RangedPullGossipService']
my_serv_impl['RangedPullGossipService'] = p2p.GossipService



my_peer_services['malicious'] = p2p.PeerType(my_peer_services['peer'].config,
                                      {p2p.BaseConnectionManager:None,
                                       MyMaliciousGossipService: my_gossip_config}
                                     )

In [18]:
print(type(my_topology))
def eclispe_peer(one_topology):
    print("eclispe_peer")
#     my_G = nx.get_nodes(one_topology)
    chosen_node = None
    chosen_neighbor = []
    for node in my_topology.nodes():
        print(node)
        for neighbor in my_topology.neighbors(node):
            print(node, neighbor)
            chosen_neighbor.append(neighbor)
            chosen_node = node
        if len(chosen_neighbor) != 0:
            print("break")
            break
            
    type_dict = nx.get_node_attributes(one_topology, 'type')
    for b in chosen_neighbor:
        type_dict[b] = 'malicious'
        
    nx.set_node_attributes(one_topology, type_dict, 'type')

eclispe_peer(my_topology)
    


<class 'networkx.classes.graph.Graph'>
eclispe_peer
1
1 2
1 4
1 5
1 6
1 9
1 14
1 20
1 22
1 25
break


In [19]:
my_sim = p2p.BaseSimulation(my_locations, my_topology, my_peer_services, my_serv_impl)
my_sim.run(3_200)

In [20]:
import pandas as pd

def message_data(sim, peer_id, storage_name):
    store = sim.peers[peer_id].storage[storage_name].txs
    for msg_id, tx in store.items():
        client_id, msg_num = msg_id.split('_')
        client_tx = sim.peers[int(client_id)].storage[storage_name].txs[msg_id]
        yield (int(msg_num), tx.data == client_tx.data)
        
def get_gossip_table(sim, storage_name, func):
    return pd.DataFrame({k: dict(func(sim, k, storage_name)) 
                         for k in set(sim.types_peers['peer'])}).sort_index()


In [21]:
my_df = get_gossip_table(my_sim, 'msg_data', message_data)
my_df

1     3     7     10    11    12    13    16    17    18    19    21  \
1  NaN  True  True  True  True  True  True  True  True  True  True  True   
2  NaN  True  True  True  True   NaN  True  True   NaN  True  True  True   
3  NaN  True   NaN  True   NaN  True  True  True  True  True  True  True   
4  NaN  True  True  True  True  True  True  True  True  True  True  True   
5  NaN  True  True  True  True  True  True  True  True   NaN  True  True   
6  NaN  True  True  True  True  True  True  True  True   NaN  True  True   
7  NaN  True   NaN   NaN   NaN  True  True   NaN  True  True   NaN  True   
8  NaN  True  True   NaN   NaN   NaN  True  True   NaN  True  True   NaN   
9  NaN  True  True  True  True   NaN  True  True   NaN  True  True  True   
10 NaN   NaN  True  True  True   NaN  True   NaN  True  True   NaN  True   
11 NaN   NaN   NaN  True   NaN   NaN   NaN   NaN   NaN  True   NaN  True   

      23    24  
1   True  True  
2   True  True  
3   True  True  
4   True   NaN  
5   True   NaN  
6    NaN  True  
7    NaN   NaN  
8    NaN   NaN  
9    NaN  True  
10   NaN  True  
11   NaN  True

In [22]:
my_df[my_df==False].count()

1     0
3     0
7     0
10    0
11    0
12    0
13    0
16    0
17    0
18    0
19    0
21    0
23    0
24    0
dtype: int64